# NR and HPC

Full nonlinear three dimensional simulations without symmetries are horribly expensive, as the CFL limit means the cost scales as $N^4$. This requires using high performance computing (HPC) resources and more complex numerical methods, particularly mesh refinement.

The key aspect of HPC is parallelism: multiple compute *cores* doing calculations at the same time. This breaks the problem into smaller pieces, solved independently, and then recombined. The problem is that the different parts of the problem need to communicate with each other, and this is much (much!) more expensive than the calculations themselves. This is the *communication* bottleneck.

For practical purposes, this means that numerical schemes are designed to run on a single *patch* of the mesh. The scheme cannot make any assumptions about how to get data from other patches. It is possible that the zones at the edge of the patch (ghost zones) are filled by the physical boundary conditions (say analytically), or by mesh refinement (say by interpolating from other data), or by data from a different patch on a different core.

## Classes

In computing terms, particularly *object oriented* computing, we have objects or *classes* which collect data and functions that act on that data as a "single thing". This helps organize the code and reduces mistakes.

A single patch would contain where it sits in the domain, its size, how the boundary zones are filled, and the data itself. The patch could also contain the functions that act on the data, such as the time stepping scheme.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipympl

First define a simple `problem` class. This will contain the number of variables and their names (useful for producing plots), along with the right-hand-side function that defines the evolution of the system. For now, it will contain no additional functions.

A python class needs to have an `__init__` function that says how to create an *instance* of the class. In this case it just saves the data on the instance. So, for example, if we were to define an instance of the problem class for advection using
```python
advection = problem(1, [r"$q$"], advection_rhs)
```
then we could access the number of variables with `advection.nv`.

In [ ]:
class problem(object):
    def __init__(self, nv, names, rhs):
        self.nv = nv
        self.names = names
        self.rhs = rhs


Now we can define a `patch` class. This will be much more complex. It will store the `problem` being solved on the patch, along with information about the size of the patch (number of interior points `nx`, number of ghost zones `ngz`, the location of the left edge of the domain `x0`, and the grid spacing `dx`). It also needs to know what physical boundary conditions are to be applied (here we only consider periodic boundaries).

Finally there is information necessary for mesh refinement; whether this is a child patch of a `parent` patch, and if so what index of the `parent` patch is linked to the first (interior) cell of this patch.

After initialisation we include some functions (*methods*) that act on a patch. These include a function to explicitly construct the coordinates of the patch (`grid`), functions to apply physical and mesh refinement boundary conditions, a function that applies a single step of Euler's method to update the data, and a simple function that plots the data on the patch.

In [ ]:

class patch(object):
    def __init__(self, problem, nx, ngz, x0, dx, bcs, parent, parent_i_start):
        self.problem = problem
        self.nx = nx
        self.ngz = ngz
        if bcs == "Periodic":
            self.bcs = self.bcs_periodic
        else:
            raise NotImplementedError(f"Boundary condition {bcs} not implemented")
        self.x0 = x0
        self.dx = dx
        self.parent = parent
        self.parent_i_start = parent_i_start
        self.data = np.zeros((problem.nv, nx+2*ngz))

    def grid(self):
        return np.linspace(self.x0-(self.ngz-0.5)*self.dx, self.x0+(self.nx+self.ngz-0.5)*self.dx, self.nx+2*self.ngz)
    
    def bcs_periodic(self):
        self.data[:, :self.ngz] = self.data[:, -2*self.ngz:-self.ngz]
        self.data[:, -self.ngz:] = self.data[:, self.ngz:2*self.ngz]
        return self.data
    
    def bcs_mr(self):
        i_s = self.parent_i_start
        i_e = self.parent_i_start + self.ngz//2
        for i in range(self.ngz//2):
            self.data[:, 2*i  ] = 0.25*(self.parent.data[:, i_s-2] + 3*self.parent.data[:, i_s-1])
            self.data[:, 2*i+1] = 0.25*(3*self.parent.data[:, i_s-1] + self.parent.data[:, i_s])
            self.data[:, -2*i-1] = 0.25*(3*self.parent.data[:, i_e+1] + self.parent.data[:, i_e+2])
            self.data[:, -2*i-2] = 0.25*(self.parent.data[:, i_e] + 3*self.parent.data[:, i_e+1])
        return self.data
    
    def restrict_interior(self):
        if self.parent is None:
            return
        for i in range(self.nx//2):
            self.parent.data[:, self.parent_i_start+i] = 0.5 * (self.data[:, 2*i+self.ngz] + self.data[:, 2*i+1+self.ngz])

    def euler_step(self, dt):
        self.data += dt * self.problem.rhs(self.data, self.dx)
        if self.parent is None:
            self.data = self.bcs()
        else:
            self.data = self.bcs_mr()
    
    def plot(self, nvs=None):
        if nvs is None:
            nvs = range(self.problem.nv)
        fig, axes = plt.subplots(len(nvs), 1, sharex=True)
        if len(nvs) == 1:
            axes = [axes]
        for i, ax in enumerate(axes):
            ax.plot(self.grid(), self.data[nvs[i], :])
            ax.set_ylabel(self.problem.names[nvs[i]])
            ax.set_xlim(self.x0, self.x0 + self.dx * self.nx)
        axes[-1].set_xlabel(r'$x$')
        fig.tight_layout()
        plt.show()

Now we can set up and evolve a problem, using advection of a sine wave as an example.

In [ ]:
def flux_advection(q):
    v = 1.0
    return v*q

def RHS_advection(q, dx):
    dqdt = np.zeros_like(q)
    dqdt[0, 1:-1] = -1.0 / (2.0*dx)*(flux_advection(q[0, 2:]) - flux_advection(q[0, :-2]))
    return dqdt

advection = problem(1, [r'$q$'], RHS_advection)
p1 = patch(advection, 100, 1, 0.0, 0.01, "Periodic", None, None)
x = p1.grid()
p1.data[0, :] = np.sin(2.0*np.pi*x)
p1.plot()
dt = 0.5 * p1.dx
t = 0
for i in range(10):
    t += dt
    p1.euler_step(dt)
    print(f"t = {t:.2f}")
p1.plot()

## Mesh refinement

So far this has added a lot of complexity for no obvious benefit. However, now we can do mesh refinement. 

One way around the computational cost problem is not to use a uniform grid. By putting grid points only where they are needed (where the solution is changing rapidly), we can reduce the number of grid points needed to get a given accuracy.

However, moving individual grid points has its own costs: it reduces accuracy and increases expense. Instead, it is easier to use *multiple* grids with different resolutions, and to move information between them. This is called *mesh refinement*.

We could imagine using a coarse grid with $N$ points covering the whole domain, and a fine grid with $N$ points with half the grid spacing covering a region where the solution changes a lot. The *effective resolution* of the interesting part of the domain is then $\Delta x / 2$, but the total computational cost is $2 N^2 + N^2 = 3 N^2$, rather than $(2 N)^2 = 4 N^2$ (the additional factor 2 in the first case is because the refined grid still has to take $2N$ timesteps due to the CFL limit). In higher dimensions, or with smaller refined grids, or with more refined grids, the savings are even greater.

The computational issue as how information is shared between grids. The refined grid needs to get its boundary conditions by interpolating from the fine grid. The coarse grid needs its interior to be updated using the (more accurate) solution from the refined grid.

We define three patches with differing grid spacings.

In [ ]:
p2 = patch(advection, 100, 1, 0.25, 0.005, "Periodic", p1, 26)
p3 = patch(advection, 100, 1, 0.375, 0.0025, "Periodic", p2, 26)

for p in [p1, p2, p3]:
    x = p.grid()
    p.data[0, :] = np.exp(-1000*(x-0.5)**2)
    p.plot()

dt = 0.5 * p3.dx
t = 0
for i in range(10):
    t += dt
    for p in [p1, p2, p3]:
        p.euler_step(dt)
    for p in [p3, p2, p1]:
        p.restrict_interior()
    print(f"t = {t:.2f}")

for p in [p1, p2, p3]:
    p.plot()

Then we generalize this approach, defining a class that will evolve arbitrary numbers of grids on arbitrary numbers of levels, and then do a sensible plot of the solution.

In [ ]:
class mr_simulation(object):
    def __init__(self, problem, base_dx, base_nx, nlevels, x_centres, child_nx):
        self.problem = problem
        p_base = patch(problem, base_nx, 1, 0.0, base_dx, "Periodic", None, None)
        levels = [[p_base]]
        for i in range(1, nlevels):
            dx = base_dx / (2**i)
            level = list()
            for j, xc in enumerate(x_centres):
                p = patch(problem, child_nx, 1, xc-0.5*child_nx*dx, dx, "Periodic", levels[i-1][j], 
                          child_nx//4+1)
                level.append(p)
            levels.append(level)
        self.levels = levels
    
    def initial_data(self, width=0.03):
        for level in self.levels:
            for p in level:
                x = p.grid()
                p.data[0, :] = np.exp(-((x-0.5)/width)**2)

    def evolve(self, t_end):
        dt = 0.5 * self.levels[-1][0].dx
        t = 0
        while t < t_end:
            t += dt
            for level in self.levels:
                for p in level:
                    p.euler_step(dt)
            for level in self.levels[::-1]:
                for p in level:
                    p.restrict_interior()
            # print(f"t = {t:.2f}")

    def plot(self, nvs=None):
        if nvs is None:
            nvs = range(self.problem.nv)
        fig, axes = plt.subplots(len(nvs), 1, sharex=True)
        if len(nvs) == 1:
            axes = [axes]
        for i, ax in enumerate(axes):
            for level in self.levels:
                for p in level:
                    ax.plot(p.grid(), p.data[nvs[i], :], 'x')
            ax.set_ylabel(self.problem.names[nvs[i]])
            ax.set_xlim(self.levels[0][0].x0, self.levels[0][0].x0 + self.levels[0][0].dx * self.levels[0][0].nx)
        axes[-1].set_xlabel(r'$x$')
        fig.tight_layout()
        plt.show()

Now try the same evolution again:

In [ ]:
mr1 = mr_simulation(advection, 0.01, 100, 3, [0.5], 100)
mr1.initial_data(width=0.03)
mr1.evolve(0.05)
mr1.plot()

Now try an evolution with a narrower Gaussian but more refinement levels.

In [ ]:
mr2 = mr_simulation(advection, 0.01, 100, 6, [0.5], 100)
mr2.initial_data(width=0.005)
mr2.evolve(0.0005)
mr2.plot()

The evolutions have been kept short as there are problems when steep features go through mesh refinement boundaries. This can be dealt with either by using better boundary conditions (*refluxing*), or by moving the meshes (*adaptive mesh refinement*). Both are more complex than this simple example. For single black holes or neutron stars, this level of complexity is enough to get started.